# Распознавание изображений цифр

In [33]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [34]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [35]:
# Нормируем входные данные в диапазон от 0 до 1
x_train = x_train / 255
x_test = x_test / 255

In [36]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [37]:
# Вытяним данные в один единый вектор
x_train = tf.reshape(tf.cast(x_train, tf.float32), [-1, 28*28])
x_test = tf.reshape(tf.cast(x_test, tf.float32), [-1, 28*28])

In [38]:
print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


In [39]:
# Преобразуем метку в one-hot вектор
y_train = to_categorical(y_train, 10)

In [40]:
y_train.shape

(60000, 10)

In [41]:
class DenseNN(tf.Module):
    def __init__(self, outputs, activate="relu"):
        super().__init__()
        self.outputs = outputs
        self.activate = activate
        self.fl_init = False

    def __call__(self, x):
        if not self.fl_init:
            self.w = tf.random.truncated_normal((x.shape[-1], self.outputs), stddev=0.1, name="w")
            self.b = tf.zeros([self.outputs], dtype=tf.float32, name='b')

            self.w = tf.Variable(self.w)
            self.b = tf.Variable(self.b)

            self.ft_init = True

        y = x @ self.w + self.b
        
        if self.activate == "relu":
            return tf.nn.relu(y)
        elif self.activate == "softmax":
            return tf.nn.softmax(y)

        return y

In [42]:
# Создаем слои:
layer_1 = DenseNN(128)
layer_2 = DenseNN(10, activate="softmax")

In [43]:
# Функцпя, которая будет проепускать вектор x через нейронную сеть
def model_predict(x):
    y = layer_1(x)
    y = layer_2(y)
    return y

In [44]:
# Задаем функцию потерь
cross_entropy = lambda y_true, y_pred: tf.reduce_mean(tf.losses.categorical_crossentropy(y_true, y_pred))

In [55]:
# Оптимизатор
opt = tf.optimizers.Adam(learning_rate=0.001)

In [56]:
BATCH_SIZE = 32
EPOCHS = 10
TOTAL = x_train.shape[0]

In [57]:
# Разобьем обучащую выборку на батчи:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

In [58]:
# Цикл обучения:


for n in range(EPOCHS):
    loss = 0
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            f_loss = cross_entropy(y_batch, model_predict(x_batch))
        loss += f_loss    
        grads = tape.gradient(f_loss, [layer_1.trainable_variables, layer_2.trainable_variables])
        opt.apply_gradients(zip(grads[0], layer_1.trainable_variables))
        opt.apply_gradients(zip(grads[1], layer_2.trainable_variables))

    print(loss.numpy()) 

ValueError: Unknown variable: <tf.Variable 'Variable:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>. This optimizer can only be called for the variables it was originally built with. When working with a new set of variables, you should recreate a new optimizer instance.

In [ ]:
y = model_predict(x_test)
y2 = tf.argmax(y, axis=1).numpy()
acc = len(y_test[y_test == y2])/y_test.shape[0] * 100
print(acc)

In [ ]:
acc = tf.metrics.Accuracy()
acc.update_state(y_test, y2)
print(acc.result().numpy() * 100)